In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('../Health_cleaned_income_delta.csv')
df.head()

,hhidpn,rmstat,ragender,rahispan,raracem,riwbegy,ragey_b,sagey_b,rhltc,rhlthlm,...,sipena_delta,rgov_delta,sgov_delta,total_work_income_delta,total_pension_income_delta,total_gov_income_delta,total_income_delta,broken,marriage_group,age_group
0,22861040,1,2,0,2,1992-04-01,62,56,-2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle
1,25154020,1,2,0,2,1992-04-01,56,59,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,married,middle
2,25181010,1,2,0,1,1992-04-06,58,53,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle
3,11067020,1,2,0,1,1992-04-07,53,50,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle
4,22861010,1,1,0,2,1992-04-07,56,62,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle


In [3]:
df['riwbegy'] = pd.to_datetime(df['riwbegy'])

In [4]:
def count_jobs(row):
    job1 = not (row['rjweeks'] == '.m' or row['rjweeks'] == '0')
    job2 = not (row['rjweek2'] == '.m' or row['rjweek2'] == '0')

    if job1 and job2:
        return 2
    elif job1 or job2:
        return 1
    return 0


df['n_jobs'] = df.apply(count_jobs, axis=1)

In [5]:
df.sort_values('riwbegy', inplace=True)
df.head()

,hhidpn,rmstat,ragender,rahispan,raracem,riwbegy,ragey_b,sagey_b,rhltc,rhlthlm,...,sipena_delta,rgov_delta,sgov_delta,total_work_income_delta,total_pension_income_delta,total_gov_income_delta,total_income_delta,broken,marriage_group,age_group
0,22861040,1,2,0,2,1992-04-01,62,56,-2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle
1,25154020,1,2,0,2,1992-04-01,56,59,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,married,middle
2,25181010,1,2,0,1,1992-04-06,58,53,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle
3,11067020,1,2,0,1,1992-04-07,53,50,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle
4,22861010,1,1,0,2,1992-04-07,56,62,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle


In [6]:
cpi_by_year = df.groupby('year')['cpi'].mean()
cpi_by_year

year
1991    0.811126
1992    0.831578
1993    0.852219
1994    0.870152
1995    0.891173
1996    0.915049
1997    0.934777
1998    0.947672
1999    0.967805
2001    1.028097
2002    1.044544
2003    1.068196
2004    1.096848
2005    1.133913
2006    1.170722
2007    1.204051
2008    1.250222
2009    1.245748
2010    1.266227
2011    1.306293
2012    1.333220
Name: cpi, dtype: float64

In [7]:
unique_ids = df['hhidpn'].unique()
unique_ids.shape

(31532,)

In [8]:
# df.rename(columns={'total_work_earn': 'total_work_income', 'total_pension_earn': 'total_pension_income'}, inplace=True)

In [9]:
df.columns

Index(['hhidpn', 'rmstat', 'ragender', 'rahispan', 'raracem', 'riwbegy',
       'ragey_b', 'sagey_b', 'rhltc', 'rhlthlm', 'rhibpe', 'rdiabe', 'rcancre',
       'rlunge', 'rhearte', 'rstroke', 'rpsyche', 'rarthre', 'rhosp',
       'rhspnit', 'oop_spend', 'riearn', 'ripena', 'siearn', 'sipena', 'rcovr',
       'rcovs', 'rlbrf', 'rjphys', 'rjlift', 'rjweeks', 'rjweek2', 'rjcten',
       'index_wave', 'insured_gov', 'uninsured', 'retired', 'collegeplus',
       'year', 'cpi', 'rgov', 'sgov', 'inter_year', 'year_of_birth',
       'total_work_income', 'total_pension_income', 'total_gov_income',
       'total_income', 'without_work', 'n_jobs', 'riearn_delta',
       'siearn_delta', 'ripena_delta', 'sipena_delta', 'rgov_delta',
       'sgov_delta', 'total_work_income_delta', 'total_pension_income_delta',
       'total_gov_income_delta', 'total_income_delta', 'broken',
       'marriage_group', 'age_group'],
      dtype='object')

In [10]:
income_columns = ['riearn', 'siearn', 'ripena', 'sipena', 'rgov', 'sgov', 'total_work_income',
        'total_pension_income', 'total_gov_income', 'total_income']

In [11]:
df.head()

,hhidpn,rmstat,ragender,rahispan,raracem,riwbegy,ragey_b,sagey_b,rhltc,rhlthlm,...,sipena_delta,rgov_delta,sgov_delta,total_work_income_delta,total_pension_income_delta,total_gov_income_delta,total_income_delta,broken,marriage_group,age_group
0,22861040,1,2,0,2,1992-04-01,62,56,-2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle
1,25154020,1,2,0,2,1992-04-01,56,59,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,married,middle
2,25181010,1,2,0,1,1992-04-06,58,53,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle
3,11067020,1,2,0,1,1992-04-07,53,50,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle
4,22861010,1,1,0,2,1992-04-07,56,62,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle


In [12]:
df['rmstat'].value_counts()

1     91489
7     28103
1     16581
5     13361
8      5091
3      4842
5      2304
4      2274
7      1913
2      1126
6      1056
3       906
8       864
4       591
2       138
.m      114
6         8
Name: rmstat, dtype: int64

In [13]:
def count_deltas(rows):
    income_deltas = {}

    for col in income_columns:
        income_deltas[f'{col}_delta'] = [np.nan]

    indexes = rows.index

    for i, (_, row) in zip(range(len(indexes)), rows.iterrows()):
        if i == 0:
            continue
        prev_row = rows.loc[indexes[i - 1]]

        valid_interval = True
        current_year = row['year']
        if current_year - prev_row['year'] > 2:
            valid_interval = False

        for col in income_columns:
            if not valid_interval or prev_row[col] == 0 or row[col] == 0:
                income_deltas[f'{col}_delta'].append(np.nan)
            else:
                income_deltas[f'{col}_delta'].append(row[col] / prev_row[col] * 100 * cpi_by_year.loc[current_year]  - 100)

    return income_deltas

In [14]:
def check_if_broken(rows):
    indexes = rows.index
    broken = False
    was_married = False

    statuses = rows['rmstat']

    for st in statuses:
        if st == '.m' or int(st) not in [1, 2, 5, 7]:
            broken = False
            break
            
        if int(st) == 1:
            was_married = True

        if int(st) in [2, 5, 7]:
            if was_married:
                broken = True
            else:
                broken = False
                break
    
        if broken and int(st) not in [2, 5, 7]:
            broken = False
            break

    return int(broken)

In [15]:
broken = []

for uid in unique_ids:
    result = check_if_broken(df[df['hhidpn'] == uid])

    df.loc[df['hhidpn'] == uid, 'broken'] = result

df.head()

,hhidpn,rmstat,ragender,rahispan,raracem,riwbegy,ragey_b,sagey_b,rhltc,rhlthlm,...,sipena_delta,rgov_delta,sgov_delta,total_work_income_delta,total_pension_income_delta,total_gov_income_delta,total_income_delta,broken,marriage_group,age_group
0,22861040,1,2,0,2,1992-04-01,62,56,-2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle
1,25154020,1,2,0,2,1992-04-01,56,59,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,married,middle
2,25181010,1,2,0,1,1992-04-06,58,53,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle
3,11067020,1,2,0,1,1992-04-07,53,50,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle
4,22861010,1,1,0,2,1992-04-07,56,62,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle


In [16]:
df[df['hhidpn'] == 15834010]

,hhidpn,rmstat,ragender,rahispan,raracem,riwbegy,ragey_b,sagey_b,rhltc,rhlthlm,...,sipena_delta,rgov_delta,sgov_delta,total_work_income_delta,total_pension_income_delta,total_gov_income_delta,total_income_delta,broken,marriage_group,age_group
2870,15834010,5,1,0,1,1992-07-13,55,.m,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,divorced,middle
17548,15834010,5,1,0,1,1994-08-20,57,.m,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,divorced,middle
23257,15834010,1,1,0,1,1996-06-15,58,46,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle
45180,15834010,1,1,0,1,1998-09-15,61,48,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle
63473,15834010,1,1,0,1,2000-08-15,63,50,-1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle
68649,15834010,1,1,0,1,2002-05-15,64,52,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle
95933,15834010,1,1,0,1,2004-08-15,67,54,0,.m,...,NaN,NaN,NaN,59.012362,NaN,NaN,89.773028,0,married,middle
110258,15834010,5,1,0,1,2006-07-15,69,.m,1,1,...,NaN,2.821062,NaN,-50.273609,NaN,2.821062,-41.667394,0,divorced,middle
126638,15834010,5,1,0,1,2008-07-15,71,.m,1,1,...,NaN,16.730065,NaN,-16.006433,NaN,16.730065,-6.653148,0,divorced,middle
137956,15834010,5,1,0,1,2010-08-15,73,.m,0,1,...,NaN,28.332463,NaN,-75.918982,NaN,28.332463,-38.671485,0,divorced,middle


In [17]:
df[df['broken'] == True].groupby('hhidpn')['rmstat'].unique()

hhidpn
10004040        [1, 1, 7]
10075020        [1, 1, 7]
10109030           [1, 7]
10210020        [1, 1, 7]
10395020     [1, 1, 7, 7]
                 ...     
918529020          [1, 2]
919258010          [1, 5]
919258020          [1, 5]
920538010          [1, 2]
920538020          [1, 2]
Name: rmstat, Length: 3480, dtype: object

In [18]:
for uid in unique_ids:
    break
    income_deltas = count_deltas(df[df['hhidpn'] == uid])

    for col in income_columns:
        df.loc[df['hhidpn'] == uid, f'{col}_delta'] = income_deltas[f'{col}_delta']

In [19]:
df[df['hhidpn'] == 22861010][income_columns + [f'{col}_delta' for col in income_columns]]

,riearn,siearn,ripena,sipena,rgov,sgov,total_work_income,total_pension_income,total_gov_income,total_income,riearn_delta,siearn_delta,ripena_delta,sipena_delta,rgov_delta,sgov_delta,total_work_income_delta,total_pension_income_delta,total_gov_income_delta,total_income_delta
4,23485.564002,0.0,0.0,0.0,0.0000,3522.8347,23485.564002,0.0,3522.8347,27008.398702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11021,25280.287153,0.0,0.0,0.0,0.0000,3768.0933,25280.287153,0.0,3768.0933,29048.380453,-8.265564,NaN,NaN,NaN,NaN,-8.844943,-8.265564,NaN,-8.844943,-8.341135
25453,23157.354808,0.0,0.0,0.0,0.0000,5252.3936,23157.354808,0.0,5252.3936,28409.748408,-18.366358,NaN,NaN,NaN,NaN,24.221806,-18.366358,NaN,24.221806,-12.841913
38761,25473.632823,0.0,0.0,0.0,0.0000,10436.9110,25473.632823,0.0,10436.9110,35910.543823,2.827702,NaN,NaN,NaN,NaN,85.747472,2.827702,NaN,85.747472,18.157907
54930,9841.727323,0.0,0.0,0.0,8843.4951,12653.6490,9841.727323,0.0,21497.1441,31338.871423,-62.608905,NaN,NaN,NaN,NaN,17.336072,-62.608905,NaN,99.340952,-15.540382
74948,19852.633829,0.0,0.0,0.0,8338.1064,7279.2988,19852.633829,0.0,15617.4052,35470.039029,107.386738,NaN,NaN,NaN,-3.065657,-40.856375,107.386738,NaN,-25.310030,16.362352
97053,0.000000,0.0,0.0,0.0,8177.9585,7197.1133,0.000000,0.0,15375.0718,15375.071800,NaN,NaN,NaN,NaN,4.767992,5.613624,NaN,NaN,5.162142,-53.697266
102817,0.000000,0.0,0.0,0.0,8028.0000,29991.3520,0.000000,0.0,38019.3520,38019.352000,NaN,NaN,NaN,NaN,11.312073,372.517084,NaN,NaN,180.393130,180.393130
121847,0.000000,0.0,0.0,0.0,7955.8916,10566.4180,0.000000,0.0,18522.3096,18522.309600,NaN,NaN,NaN,NaN,19.323598,-57.579421,NaN,NaN,-41.340916,-41.340916
140705,0.000000,0.0,0.0,0.0,0.0000,8585.2588,0.000000,0.0,8585.2588,8585.258800,NaN,NaN,NaN,NaN,NaN,1.217546,NaN,NaN,-42.258448,-42.258448


In [34]:
def reformat_rhltc(el):
    if el == '.m':
        return el
    return 3 - int(el)


def return_group(el):
    if el == '.m':
        return '.m'
    if int(el) == 1 or int(el) == 3:
        return 'married'
    if int(el) == 4 or int(el) == 6:
        return 'separated'
    if int(el) == 2 or int(el) == 7:
        return 'dead spouse'
    if int(el) == 8:
        return 'never married'
    if int(el) == 5:
        return 'divorced'


def return_age_group(el):
    age = int(el)
    if age < 45:
        return 'young'
    elif 45 <= age <= 65:
        return 'middle'
    elif 65 <= age <= 80:
        return 'old'
    else:
        return 'dead outside'

df['rhltc'] = df['rhltc'].apply(reformat_rhltc)
df['marriage_group'] = df['rmstat'].apply(return_group)
df['age_group'] = df['ragey_b'].apply(return_age_group)

In [28]:
df.head()

,hhidpn,rmstat,ragender,rahispan,raracem,riwbegy,ragey_b,sagey_b,rhltc,rhlthlm,...,sipena_delta,rgov_delta,sgov_delta,total_work_income_delta,total_pension_income_delta,total_gov_income_delta,total_income_delta,broken,marriage_group,age_group
0,22861040,1,2,0,2,1992-04-01,62,56,-2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle
1,25154020,1,2,0,2,1992-04-01,56,59,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,married,middle
2,25181010,1,2,0,1,1992-04-06,58,53,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle
3,11067020,1,2,0,1,1992-04-07,53,50,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle
4,22861010,1,1,0,2,1992-04-07,56,62,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,married,middle


In [33]:
df['age_group'].value_counts().reindex(['young', 'middle', 'old', 'dead outside'])

young            32770
middle          110607
old              25655
dead outside      1729
Name: age_group, dtype: int64

In [31]:
df['age_group'].value_counts().reindex(['young', 'middle', 'old', 'dead outside'])

young            1495
middle          92819
old             57009
dead outside    19438
Name: age_group, dtype: int64

In [35]:
df.to_csv('../Health_cleaned_income_delta.csv', index=False)